# Multi-Agent Orchestration with Semantic Kernel

Welcome to the fascinating world of multi-agent orchestration! In this notebook, we'll explore how multiple AI agents can work together to solve complex problems through different orchestration patterns.



## Package References and Configuration

First, let's install the required NuGet packages for multi-agent orchestration and create our kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1

## Configuration and Settings

Load our AI configuration settings:

## Using Statements and Dependencies

Import all necessary namespaces for multi-agent orchestration:

In [2]:
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview"

Installed Packages Microsoft.SemanticKernel.Agents.Core, 1.67.1 Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview

because we are using a preview and (still) experimental version, we need to diable the warning:
*error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.Concurrent.ConcurrentOrchestration' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.*

In [3]:
#pragma warning disable SKEXP0110

In [4]:
// Core Semantic Kernel imports
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// System imports for orchestration
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.Threading;
using System.Linq;
using System.Collections.Concurrent;
using System.Text;
using Microsoft.DotNet.Interactive;

// Kernel alias to avoid conflicts
using Kernel = Microsoft.SemanticKernel.Kernel;

Console.WriteLine("All dependencies imported successfully! 🎯");

All dependencies imported successfully! 🎯


## Helper functions



In [5]:
// Helper function to display agent responses
void DisplayResponse(string agentName, string response, string emoji = "🤖")
{
    Console.WriteLine($"\n{emoji} **{agentName}:**");
    Console.WriteLine($"   {response}");
    Console.WriteLine(new string('-', 50));
}

// Helper function to display section headers
void DisplaySection(string title, string emoji = "🎯")
{
    Console.WriteLine($"\n{emoji} {title}");
    Console.WriteLine(new string('=', title.Length + 4));
}

Console.WriteLine("🛠️ Helper functions created successfully!");

🛠️ Helper functions created successfully!


## Simple Concurrent Orchestration

Let's start with our first orchestration pattern: **Concurrent Orchestration**

### Scenario: Restaurant Kitchen Team 🍽️

Imagine a busy restaurant kitchen where multiple chefs work simultaneously on different parts of a meal:

In [6]:
#pragma warning disable SKEXP0110

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Orchestration.Concurrent;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;


DisplaySection("CONCURRENT ORCHESTRATION - Restaurant Kitchen Team", "🍽️");

// Create our restaurant kitchen team
var sousChef = new ChatCompletionAgent
    {
        Name = "SousChef",
        Description = "You are a sous chef specializing in appetizers and salads",
        Instructions = "Be creative and describe the dish preparation briefly",
        Kernel = kernel,
        Arguments = new KernelArguments(new OpenAIPromptExecutionSettings
        {
            MaxTokens = 500,
            Temperature = 0.7
        })
    };

var mainChef = new ChatCompletionAgent
    {
        Name = "MainChef",
        Description = "You are a main course chef specializing in proteins and hearty dishes",
        Instructions = "Be creative and describe the cooking process briefly",
        Kernel = kernel
    };


var pastryChef = new ChatCompletionAgent
    {
        Name = "PastryChef",
        Description = "You are a pastry chef specializing in desserts and sweet treats",
        Instructions = "Be creative and describe the dessert preparation briefly",
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef);

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestration.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();


🍽️ CONCURRENT ORCHESTRATION - Restaurant Kitchen Team

# RESULT:
Sure! Here's a delightful Israeli meal that’s vibrant, fresh, and full of bold flavors:

**Dish Name:** *Sabich Platter with Fresh Sides*

**Main Dish: Sabich Pita**  
1. Start by roasting thin slices of eggplant until they’re golden and creamy inside.  
2. Boil eggs to a jammy perfection (around 7 minutes for that soft center).  
3. Warm fluffy pita bread on a skillet for a slight char.  
4. Assemble: Slather the pita with a generous spoonful of tahini, layer in the roasted eggplant, sliced eggs, crisp cucumber-tomato salad, tangy pickles, and a drizzle of amba (a spiced mango sauce). Sprinkle with fresh parsley.

**Side 1: Israeli Salad**  
1. Dice cucumbers and ripe tomatoes into small cubes.  
2. Toss with fresh parsley, a squeeze of lemon juice, olive oil, and a pinch of salt. This refreshing side complements the richness of the Sabich.

**Side 2: Hummus with Za’atar Oil**  
1. Blend cooked chickpeas with tahini, ga

### Adding the inner interaction and handling messages

In [7]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var orchestrationWithInnerCommunication =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestrationWithInnerCommunication.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
//Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();




🤖 **MainChef:**
   Sure, let's make a scrumptious Israeli meal featuring *Shakshuka* with a side of *Israeli Salad* and fresh pita bread!

---

**Shakshuka**  
1. Heat olive oil in a wide, deep skillet over medium heat. Sauté diced onions, garlic, and bell peppers until soft and golden.  
2. Add a generous spoon of tomato paste, followed by crushed canned tomatoes (or fresh ones if you prefer). Season with paprika, cumin, chili flakes, salt, pepper, and a pinch of sugar to balance the acidity. Let it simmer until thickened.  
3. Gently crack eggs into small pockets you create in the bubbly tomato sauce. Cover the pan and allow the eggs to cook just until whites are set but yolks are still runny. Sprinkle chopped parsley or cilantro for a burst of flavor and color.  

---

**Israeli Salad**  
1. Finely chop cucumbers, juicy tomatoes, red onion, and fresh parsley.  
2. Drizzle with olive oil, freshly squeezed lemon juice, and a pinch of salt and pepper. Mix together for the ultimate ref

## Sequential Orchestration

Now let's explore **Sequential Orchestration** where agents work in a specific order.

### Scenario: News Editorial Team 📰

In a newsroom, articles go through a specific workflow: Writing → Editing

In [8]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.Sequential;

DisplaySection("SEQUENTIAL ORCHESTRATION - News Editorial Team", "📰");

// Create our news editorial team
var writer = new ChatCompletionAgent
    {
        Name = "Writer",
        Description = "You are a news writer",
        Instructions = "Take the research and write a compelling news article. Keep it engaging and informative",
        Kernel = kernel
    };


var editor = new ChatCompletionAgent
    {
        Name = "Editor",
        Description = "You are a news editor",
        Instructions = 
        """
        Review the given draft article and imporve clarity, correct grammer and shorten it.
        Output the final improved copy as a single text block.
        """,
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new SequentialOrchestration(writer, editor)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Write an article on Semantic Kernel Agents", runtime);

string output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{output}");

await runtime.RunUntilIdleAsync();


📰 SEQUENTIAL ORCHESTRATION - News Editorial Team

🤖 **Writer:**
   **Semantic Kernel Agents: Revolutionizing AI with Smarter, More Human-like Interactions**

In today’s fast-evolving technological ecosystem, the challenge of creating AI systems that can process information beyond mere pattern recognition has been at the forefront of AI research. Enter Semantic Kernel Agents, a groundbreaking advancement in the field of artificial intelligence. This innovative strategy is poised to redefine how humans and machines interact by applying the power of semantic reasoning and context-awareness to practical applications, and its implications could ripple across nearly every industry.

### What Are Semantic Kernel Agents?

At a high level, Semantic Kernel Agents act as intermediaries between humans and machines, facilitating more natural interactions by integrating semantic understanding into artificial intelligence. These agents draw upon the power of semantic kernels—structured systems that 

## Group Chat Orchestration

Let's explore **Group Chat Orchestration** where agents collaborate in a discussion.

### Scenario: Medical Consultation Team 🏥

A medical consultation where specialists discuss a patient case together:

In [9]:
DisplaySection("GROUP CHAT ORCHESTRATION - Medical Consultation Team", "🏥");

// Create our medical consultation team
var generalDoctor = new ChatCompletionAgent
    {
        Name = "GeneralDoctor",
        Description = "You are a general practitioner",
        Instructions = 
        """
        Provide overall medical assessment and coordinate with specialists. 
        Be professional and concise.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };

 
var cardiologist = new ChatCompletionAgent
    {
        Name = "Cardiologist",
        Description = "You are a heart specialist",
        Instructions = 
        """
        Focus on cardiovascular aspects of the case. 
        Provide expert cardiac insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };
    
var neurologist = new ChatCompletionAgent
    {
        Name = "Nutritionist",
        Description = "You are a neurologist",
        Instructions = 
        """
        Focus on neurological aspects of the case. 
        Provide expert brain and nervous system insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };



// Create a shared chat history for the group consultation
var groupChatHistory = new ChatHistory();



🏥 GROUP CHAT ORCHESTRATION - Medical Consultation Team


In [10]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

var orchestration =
    new GroupChatOrchestration(
        new RoundRobinGroupChatManager()  { MaximumInvocationCount = 5  },
        generalDoctor,
        cardiologist,
        neurologist)
    {       
        ResponseCallback =  chatMessage => 
        { 
            groupChatHistory.Add(chatMessage);
            return ValueTask.CompletedTask;
        }
    };

// Start the runtime
var runtime = new InProcessRuntime();
await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Everytime i workout my breath makes funny sounds. Tell me what to do", runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in groupChatHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


# RESULT: Based on all the provided insights, let's consolidate everything with a primary cardiology-oriented focus, since the cardiovascular system could be a key player in your exercise-induced symptoms. Here's a concise cardiac evaluation plan:

---

### Cardiologist’s Specific Concerns:
1. **Exercise-Induced Breath Sounds** might indicate:
   - **Pulmonary Congestion Secondary to Cardiac Dysfunction**: This could happen in early heart failure or with certain valve disorders (e.g., mitral regurgitation).
   - **Arrhythmias Triggered by Exercise**: Irregular heartbeats may impair efficient blood and oxygen delivery to the lungs, causing compensatory noisy breathing.
   - **Impaired Cardiac Output or Ischemia**: During exercise, the heart may fail to meet the oxygen demand due to conditions like coronary artery disease. This, in turn, may lead to labored or noisy breathing.

---

### Action Plan for Cardiac Assessment:
1. **Diagnostic Tests**:
   - **Echocardiogram**:
     - To evalu

## Custom Orchestration

Finally, let's explore **Custom Orchestration** - controlling the agents selection and termincation logic.

### Scenario: Hospital's Tumor Board 🎵

Hospital's tumor board is reviewing a complex cancer case:

In [11]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;
using Microsoft.SemanticKernel.ChatCompletion;

class TumorBoardLeader : GroupChatManager
{
    string _boardCase;
    IChatCompletionService _chatCompletion;

    public TumorBoardLeader(string boardCase, IChatCompletionService chatCompletion)
    {
        _boardCase = boardCase;
        _chatCompletion = chatCompletion;
    }

    private static class Prompts
    {
        public static string Termination(string boardCase) =>
            $"""
                You are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to determine if the discussion has reached a conclusion and if all next steps were listed.
                if no artifacts were produced yet and you don't have knowledge of them yet, then continue the dicsussion and planning.
                
                If you would like to end the discussion, please respond with True. Otherwise, respond with False.
                """;

        public static string Selection(string boardCase, string participants) =>
            $"""
                Tou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to select the next team member to speak and suggest a plan or next steps. 
                Here are the roles and descriptions of the participants: 
                {participants}\n
                Please respond with only the role of the participant you would like to select.
                """;

        public static string Filter(string boardCase) =>
            $"""
                ou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You have just concluded the discussion and team work. 
                Please summarize the discussion and provide the final artifacts.
                """;
    }

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> FilterResults(ChatHistory history, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Filter(_boardCase), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> SelectNextAgent(ChatHistory history, GroupChatTeam team, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Selection(_boardCase, team.FormatList()), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(ChatHistory history, CancellationToken cancellationToken = default) =>
        ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "The AI group chat manager does not request user input." });

    /// <inheritdoc/>
    public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(ChatHistory history, CancellationToken cancellationToken = default)
    {
        GroupChatManagerResult<bool> result = await base.ShouldTerminate(history, cancellationToken);
        if (!result.Value)
        {
            result = await this.GetResponseAsync<bool>(history, Prompts.Termination(_boardCase), cancellationToken);
        }
        return result;
    }

    private async ValueTask<GroupChatManagerResult<TValue>> GetResponseAsync<TValue>(ChatHistory history, string prompt, CancellationToken cancellationToken = default)
    {
        OpenAIPromptExecutionSettings executionSettings = new() { ResponseFormat = typeof(GroupChatManagerResult<TValue>) };
        ChatHistory request = [.. history, new ChatMessageContent(AuthorRole.System, prompt)];
        ChatMessageContent response = await _chatCompletion.GetChatMessageContentAsync(request, executionSettings, kernel: null, cancellationToken);
        string responseText = response.ToString();
        
        return
            JsonSerializer.Deserialize<GroupChatManagerResult<TValue>>(responseText) ??
            throw new InvalidOperationException($"Failed to parse response: {responseText}");
    }
}

In [12]:
DisplaySection("CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case", "🎵");

var oncologistAgent = new ChatCompletionAgent
{
    Name = "OncologistAgent",
    Instructions = 
        """
        You are a medical oncologist participating in a multidisciplinary discussion about a cancer patient's treatment. 
        Your role is to evaluate drug-based treatment options, considering cancer stage, biomarkers, and mutation data. 
        You collaborate with a radiologist and a pathologist. Be open to revising your recommendation based on their input.
        Only suggest treatments after reviewing all available clinical and diagnostic information.

        You are the final caller, the decision comes from you and you decide if more inputs are needed from the others.
        The final call and decision is yours! the result of the discussion should come from you so when you think the discussion is over, say it.
        """,
    Description = "A medical oncologist who suggests and revises treatment plans based on pathology and imaging insights.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var pathologistAgent = new ChatCompletionAgent
{
    Name = "PathologistAgent",
    Instructions = 
        """
        You are a clinical pathologist providing molecular and genetic analysis of a patient's tumor. 
        Share insights about mutations, biomarkers, and how they might affect treatment effectiveness. 
        Collaborate with the oncologist and radiologist. Suggest when additional lab tests may be needed.
        The final call and decision is of the oncologist!
        """,
    Description = "A clinical pathologist who interprets molecular and genetic data relevant to treatment decisions.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})

};

var radiologistAgent = new ChatCompletionAgent
{
    Name = "RadiologistAgent",
    Instructions = 
        """
        You are a radiologist interpreting imaging results for a cancer patient. 
        Point out any concerns such as lymph node involvement or metastasis. 
        Inform how imaging findings affect staging and treatment planning. Collaborate with the oncologist and pathologist.
        The final call and decision is of the oncologist!
        """,
    Description = "A radiologist who provides imaging-based insights and identifies implications for treatment planning.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

// Create a shared chat history for the group consultation
var tumorBoardHistory = new ChatHistory();


🎵 CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case


In [13]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

string input =
    """
    Patient: Female, 55 years old  
    Diagnosis: Invasive ductal carcinoma, grade 3  
    Tumor Size: 2.8 cm  
    Stage: T2N0M0 (early-stage)  
    Hormone Receptors: ER+, PR+, HER2-  
    Ki-67: 35% (high proliferation)  
    Genetic Testing: BRCA-negative  
    Genomic Profile: PIK3CA mutation detected  
    Imaging: Mammogram and MRI confirm localized tumor, but one axillary lymph node appears suspicious (not biopsied yet)  
    Comorbidities: Mild hypertension  

    Please collaborate and determine the optimal treatment plan for this patient. Consider systemic therapies, genetic markers, and imaging findings. 
    Discuss whether further testing is needed, and iterate based on each other's input.
    The final call and decision is of the oncologist.
    """;


var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var tumorBoardOrchestration =
   new GroupChatOrchestration(
       new TumorBoardLeader(input, chatCompletionService),
       
       oncologistAgent, 
       pathologistAgent, 
       radiologistAgent)
   {
    ResponseCallback =  chatMessage => 
       { 
            DisplayResponse(chatMessage.AuthorName, chatMessage.Content);
            tumorBoardHistory.Add(chatMessage);
            return ValueTask.CompletedTask;  
       }
   };
// Start the runtime
InProcessRuntime runtime = new();
await runtime.StartAsync();



OrchestrationResult<string> result = await tumorBoardOrchestration.InvokeAsync(input, runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in tumorBoardHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


# RESULT: Recommendations from the Tumor Board Discussion:

1. Perform a biopsy of the suspicious axillary lymph node to confirm metastatic involvement and refine staging.

2. Considering the patient's early-stage, hormone receptor-positive (ER+/PR+), and HER2-negative tumor status:

   a. Initiate endocrine therapy with an aromatase inhibitor due to postmenopausal age group.

   b. Discuss adjuvant chemotherapy options, taking into account the high Ki-67 index (35%) and grade 3 classification, which indicate an aggressive tumor nature.

3. Based on the identified PIK3CA mutation, consider targeted therapies after primary treatment, following current guidelines and trial results.

4. Account for the patient's mild hypertension during treatment selection to minimize exacerbation of comorbid conditions.

5. Schedule regular imaging follow-ups and biomarker assessments to monitor treatment response and identify potential recurrence early.

Final decision to be executed by the treating on

## Human-in-the-Loop (HITL) with Group Chat Orchestration



Now let's explore adding **Human-in-the-Loop** to our agent orchestration.



### Scenario: Medical Team with Human Approval Gate 🏥🚦



A medical consultation where the team's recommendation requires human approval before being finalized:

In [17]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

// Custom GroupChatManager with Human Approval
class HumanApprovalManager(string lastAgentName) : RoundRobinGroupChatManager
{
    private bool _approvalRequested = false;    

    
    public override ValueTask<GroupChatManagerResult<string>> FilterResults(ChatHistory history, CancellationToken cancellationToken = default)
        {
            ChatMessageContent finalResult = history.Last(message => message.AuthorName == lastAgentName);
            return ValueTask.FromResult(new GroupChatManagerResult<string>($"{finalResult}") { Reason = "The approved result." });
        }

        /// <inheritdoc/>
        public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(ChatHistory history, CancellationToken cancellationToken = default)
        {
            // Has the maximum invocation count been reached?
            GroupChatManagerResult<bool> result = await base.ShouldTerminate(history, cancellationToken);
            if (!result.Value)
            {
                // If not, check if the reviewer has approved the copy.
                ChatMessageContent? lastMessage = history.LastOrDefault();
                if (lastMessage is not null && lastMessage.Role == AuthorRole.User && $"{lastMessage}".Contains("Yes", StringComparison.OrdinalIgnoreCase))
                {
                    // If the reviewer approves, we terminate the chat.
                    result = new GroupChatManagerResult<bool>(true) { Reason = "The user is satisfied with the copy." };
                }
            }
            return result;
        }

        public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(ChatHistory history, CancellationToken cancellationToken = default)
        {
            ChatMessageContent? lastMessage = history.LastOrDefault();

            if (lastMessage is null)
            {
                return ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "No agents have spoken yet." });
            }

            if (lastMessage is not null && lastMessage.AuthorName == lastAgentName)
            {
                return ValueTask.FromResult(new GroupChatManagerResult<bool>(true) { Reason = $"User input is needed after the {lastAgentName} message." });
            }

            return ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "User input is not needed until the reviewer's message." });
        }
}

Console.WriteLine("✅ HumanApprovalManager created!");

✅ HumanApprovalManager created!


### Run the HITL Orchestration



Now let's create a medical consultation with human approval:

In [ ]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

DisplaySection("HITL ORCHESTRATION - Medical Team with Human Approval", "🏥🚦");



// Create medical team agents

var generalDoctor = new ChatCompletionAgent
{
    Name = "GeneralDoctor",
    Description = "You are a general practitioner",
    Instructions = 
    """
    Provide overall medical assessment and coordinate with specialists. 
    Be professional and concise.
    When you receive human feedback, address it seriously and revise your recommendations.
    Summarize the input from other doctors.
    """,
    Kernel = kernel
};



var cardiologist = new ChatCompletionAgent
{
    Name = "Cardiologist",
    Description = "You are a heart specialist",

    Instructions = 

    """

    Focus on cardiovascular aspects of the case. 

    Provide expert cardiac insights.

    If human feedback mentions cardiac concerns, address them specifically.

    """,

    Kernel = kernel

};



var nutritionist = new ChatCompletionAgent

{

    Name = "Nutritionist",
    Description = "You are a nutritionist",
    Instructions = 
    """
    Focus on dietary and lifestyle aspects of the case.
    Provide nutrition-based recommendations.
    Incorporate human feedback into dietary plans.
    """,
    Kernel = kernel

};



// Create chat history

var hitlChatHistory = new ChatHistory();

bool didUserRespond = false;

// Create orchestration with human approval
var hitlOrchestration = new GroupChatOrchestration(

    new HumanApprovalManager("Nutritionist") 
    { 
        MaximumInvocationCount = 6,
        InteractiveCallback = () =>
                    {
                        // Simlulate user input that first replies "No" and then "Yes"
                        ChatMessageContent input = new(AuthorRole.User, didUserRespond ? "Yes" : "More discussion");
                        didUserRespond = true;
                        Console.WriteLine($"\n# INPUT: {input.Content}\n");
                        return ValueTask.FromResult(input);
                    } },  // Max 6 agent turns before requiring approval

    generalDoctor,
    cardiologist,
    nutritionist)
{

    ResponseCallback = chatMessage => 
    {
        DisplayResponse(chatMessage.AuthorName, chatMessage.Content);
        hitlChatHistory.Add(chatMessage);
        return ValueTask.CompletedTask;
    }
};



// Start runtime
var runtime = new InProcessRuntime();
await runtime.StartAsync();

// Run the consultation
var result = await hitlOrchestration.InvokeAsync(
    "Patient has high blood pressure and high cholesterol. Recommend a treatment plan.",
    runtime);

string finalRecommendation = await result.GetValueAsync();
await runtime.RunUntilIdleAsync();



Console.WriteLine("\n" + new string('=', 60));
Console.WriteLine("📋 FINAL APPROVED RECOMMENDATION:");
Console.WriteLine(new string('=', 60));

Console.WriteLine(finalRecommendation);


🏥🚦 HITL ORCHESTRATION - Medical Team with Human Approval
